In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
DATA_PATH = '../data/'

users = pd.read_csv(DATA_PATH + 'users.csv')
books = pd.read_csv(DATA_PATH + 'books.csv')
train = pd.read_csv(DATA_PATH + 'train_ratings.csv')
test = pd.read_csv(DATA_PATH + 'test_ratings.csv')
sub = pd.read_csv(DATA_PATH + 'sample_submission.csv')

In [6]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149570 entries, 0 to 149569
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 149570 non-null  object 
 1   book_title           149570 non-null  object 
 2   book_author          149570 non-null  object 
 3   year_of_publication  149570 non-null  float64
 4   publisher            149570 non-null  object 
 5   img_url              149570 non-null  object 
 6   language             82343 non-null   object 
 7   category             80719 non-null   object 
 8   summary              82343 non-null   object 
 9   img_path             149570 non-null  object 
dtypes: float64(1), object(9)
memory usage: 11.4+ MB


In [7]:
users['location_city'] = users['location'].apply(lambda x: x.split(',')[0])
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1])
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2])
users = users.drop(['location'], axis=1)

In [8]:
len(users['location_country'].unique())

347

In [9]:
users['location_country'].value_counts()[:30]

 usa               43991
 canada             6208
 germany            3526
 united kingdom     2914
                    2101
 australia          1791
 spain              1653
 france              794
 italy               770
 switzerland         451
 new zealand         448
 netherlands         403
 portugal            325
 austria             251
 malaysia            167
 singapore           129
 sweden              120
 ireland             107
 belgium             104
 brazil              103
 finland              89
 philippines          80
 argentina            77
 japan                74
 mexico               65
 denmark              61
 norway               54
 israel               48
 india                39
 romania              39
Name: location_country, dtype: int64

In [10]:
users['location_city'] = users['location_city'].str.strip()
users['location_state'] = users['location_state'].str.strip()
users['location_country'] = users['location_country'].str.strip()

In [11]:
users['location_city'] = users['location_city'].str.replace(r'[^a-zA-Z]', '', regex=True)
users['location_state'] = users['location_state'].str.replace(r'[^a-zA-Z]', '', regex=True)
users['location_country'] = users['location_country'].str.replace(r'[^a-zA-Z]', '', regex=True)

In [12]:
'''
location_country
'''
# null & na & universe & etc
null_repl = [
    'universe', 'na', '', 'lava', 'petrolwarnation', 'space', 'lachineternelle',
    'faraway', 'everywhereandanywhere', 'hereandthere', 'tdzimi', 'naontheroad',
    'unknown'
]
for keyword in null_repl:
    users.loc[users['location_country'] == keyword, 'location_country'] = 'null'
users.loc[users['location_country'] == 'c', 'location_country'] = 'null'
# australia
australia_repl = [
    'newsouthwales', 'queensland', 'tasmania', 'victoria', 'nsw', 'southaustralia'
]
for keyword in australia_repl:
    users.loc[users['location_country'].str.contains(keyword), 'location_country'] = 'australia'
# italy
users.loc[users['location_country'].str.contains('ital'), 'location_country'] = 'italy'
users.loc[users['location_country'].str.contains('ferrara'), 'location_country'] = 'italy'
users.loc[users['location_country'].str.contains('veneziagiulia'), 'location_country'] = 'italy'
users.loc[users['location_country'].str.contains('ineurope'), 'location_country'] = 'italy'
# germany
users.loc[users['location_country'].str.contains('deut'), 'location_country'] = 'germany'
users.loc[users['location_country'].str.contains('germ'), 'location_country'] = 'germany'
users.loc[users['location_country'].str.contains('berlin'), 'location_country'] = 'germany'
users.loc[users['location_country'].str.contains('niedersachsen'), 'location_country'] = 'germany'
# united kingdom
uk_repls = [
    'unitedkingdom', 'eng', 'king', 'wales', 'scotland', 'aberdeenshire', 'camden', 'unitedkindgonm',
    'middlesex', 'nottinghamshire', 'westyorkshire', 'cambridgeshire', 'sthelena', 'northyorkshire',
    'obviously'
]
for keyword in uk_repls:
    users.loc[users['location_country'].str.contains(keyword), 'location_country'] = 'united kingdom'
users.loc[users['location_country'] == 'uk', 'location_country'] = 'united kingdom'
# ireland
users.loc[users['location_country'].str.contains('countycork'), 'location_country'] = 'ireland'
users.loc[users['location_country'].str.contains('cocarlow'), 'location_country'] = 'ireland'
# france
users.loc[users['location_country'].str.contains('fran'), 'location_country'] = 'france'
users.loc[users['location_country'].str.contains('paris'), 'location_country'] = 'france'
# spain
spain_repl = [
    'esp', 'catal', 'galiza', 'euskalherria', 'lleida', 'gipuzkoa', 'orense', 'pontevedra', 'almera',
    'bergued', 'andalucia'
]
for keyword in spain_repl:
    users.loc[users['location_country'].str.contains(keyword), 'location_country'] = 'spain'
# portugal
users.loc[users['location_country'].str.contains('oeiras'), 'location_country'] = 'portugal'
# belgium
users.loc[users['location_country'].str.contains('labelgique'), 'location_country'] = 'belgium'
# austria
users.loc[users['location_country'].str.contains('eu'), 'location_country'] = 'austria'
# swiss
users.loc[users['location_country'].str.contains('lasuisse'), 'location_country'] = 'switzerland'
# finland
users.loc[users['location_country'].str.contains('etelsuomi'), 'location_country'] = 'finland'
# usa
usa_repl = [
    'unitedstaes', 'america', 'usa', 'state', 'sate', 'cali', 'dc', 'oregon', 'texas', 'florida',
    'newhampshire', 'newmexico', 'newjersey', 'newyork', 'virginia', 'bermuda', 'illinois', 'michigan',
    'arizona', 'indiana', 'minnesota', 'tennessee', 'dakota', 'connecticut', 'wisconsin', 'ohio',
    'maryland', 'northcarolina', 'massachusetts', 'colorado', 'washington', 'maine', 'georgia', 'oklahoma',
    'maracopa', 'districtofcolumbia', 'saintloius', 'orangeco', 'aroostook', 'arkansas', 'montana',
    'rhodeisland', 'nevada', 'kern', 'fortbend', 'nebraska', 'usofa', 'alabama', 'csa', 'polk',
    'alachua', 'austin', 'alaska', 'hawaii', 'worcester', 'iowa', 'cherokee', 'shelby', 'stthomasi',
    'vanwert', 'kansas', 'idaho', 'tn', 'framingham', 'pender', 'ysa', 'arizona', 'morgan', 'rutherford'
]
for keyword in usa_repl:
    users.loc[users['location_country'].str.contains(keyword), 'location_country'] = 'usa'
users.loc[users['location_country'] == 'us', 'location_country'] = 'usa'
users.loc[users['location_country'] == 'ca', 'location_country'] = 'usa'
users.loc[users['location_country'] == 'il', 'location_country'] = 'usa'
users.loc[users['location_country'] == 'ua', 'location_country'] = 'usa'
# cananda
canada_repl = [
    'cananda', 'british', 'newfoundland', 'newbrunswick', 'alberta', 'ontario', 'lkjlj', 'bc',
    'novascotia', 'kcb', 'quebec', 'maricopa', 'travelling', 'vvh', 'saskatchewan'
]
for keyword in canada_repl:
    users.loc[users['location_country'].str.contains(keyword), 'location_country'] = 'canada'
# new zealand
users.loc[users['location_country'] == 'nz', 'location_country'] = 'newzealand'
users.loc[users['location_country'].str.contains('otago'), 'location_country'] = 'newzealand'
users.loc[users['location_country'].str.contains('auckland'), 'location_country'] = 'newzealand'
# malaysia
users.loc[users['location_country'].str.contains('kedah'), 'location_country'] = 'malaysia'
# uae
users.loc[users['location_country'].str.contains('uae'), 'location_country'] = 'unitedarabemirates'
# kuwait
users.loc[users['location_country'].str.contains('quit'), 'location_country'] = 'kuwait'
# phillipines
users.loc[users['location_country'].str.contains('phill'), 'location_country'] = 'philippines'
users.loc[users['location_country'].str.contains('metromanila'), 'location_country'] = 'philippines'
# uruguay
users.loc[users['location_country'].str.contains('urugua'), 'location_country'] = 'uruguay'
# panama
users.loc[users['location_country'].str.contains('republicofpanama'), 'location_country'] = 'panama'
# trinidadandtobago
users.loc[users['location_country'].str.contains('westindies'), 'location_country'] = 'trinidadandtobago'
# guernsey
users.loc[users['location_country'].str.contains('alderney'), 'location_country'] = 'guernsey'
# japan
users.loc[users['location_country'].str.contains('okinawa'), 'location_country'] = 'japan'
# korea
users.loc[users['location_country'].str.contains('seoul'), 'location_country'] = 'southkorea'
# brazil
users.loc[users['location_country'].str.contains('disritofederal'), 'location_country'] = 'brazil'


'''
location_state
'''
# usa
usa_state_repl = [
    'usa', 'texas', 'tx', 'california', 'massachusetts', 'michigan', 'carolina', 'florida', 'colorado', 'pennsylvania',
    'newyork', 'newjersey', 'virginia', 'dc', 'washington', 'iowa', 'illinois', 'georgia', 'kansas', 'missouri',
    'mississippi', 'oregon', 'arizona', 'ohio', 'tennessee', 'idaho', 'alaska', 'alabama', 'minnesota', 'utah',
    'kentucky', 'rhodeisland', 'maryland', 'louisiana', 'indiana', 'connecticut', 'wisconsin', 'newhampshire',
    'nevada', 'oklahoma', 'georgia', 'maine', 'newmexico', 'nebraska', 'wyoming', 'frenchquarter', 'fl', 'nebr', 'ct',

]
for keyword in usa_state_repl:
    users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains(keyword)), 'location_country'] = 'usa'
# canada
canada_state_repl = [
    'britishcolumbia', 'newbrunswick', 'novascotia', 'ontario', 'alberta', 'quebec', 'saskatchewan',
    'manitoba', 
]
for keyword in canada_state_repl:
    users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains(keyword)), 'location_country'] = 'canada'
# mexico
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('jalisco')), 'location_country'] = 'mexico'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('michoacan')), 'location_country'] = 'mexico'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('morelos')), 'location_country'] = 'mexico'
# united kingdom
uk_state_repl = [
    'newhampshire', 'nottinghamshire', 'england', 'middlesex', 'midlothian', 'scotland', 'westyorkshire',
    'canterbury', 'wiltshire', 'kent', 'london', 'cambs', 'herts', 'isleofman', 'surrey', 'cheshire',
    'gloucestershire', 'aberdeenshire'
]
for keyword in uk_state_repl:
    users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains(keyword)), 'location_country'] = 'united kingdom'
# ireland
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('dublin')), 'location_country'] = 'ireland'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('wicklow')), 'location_country'] = 'ireland'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('colimerick')), 'location_country'] = 'ireland'
# australia
australia_state_repl = [
    'newsouthwales', 'victoria', 'australiancapitalterritory', 'southaustralia', 'nsw'
]
for keyword in australia_state_repl:
    users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains(keyword)), 'location_country'] = 'australia'
# germany
germany_state_repl = [
    'nordrheinwestfalen', 'bayern', 'hamburg', 'badenwuerttemberg', 'badenwrttemberg', 'sachsen', 'berlin',
    'stuttgart', 'nrw', 'bavaria', 'bremen'
]
for keyword in germany_state_repl:
    users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains(keyword)), 'location_country'] = 'germany'
# switzerland
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('bern')), 'location_country'] = 'switzerland'
# austria
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('niederoesterreich')), 'location_country'] = 'austria'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('vienna')), 'location_country'] = 'austria'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('wien')), 'location_country'] = 'austria'
# slovenia
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('ljubljanskaregija')), 'location_country'] = 'slovenia'
# spain
spain_state_repl = [
    'catalunya', 'pontevedra', 'madrid', 'bizkaia', 'asturias', 'pontevedra', 'barcelona', 'pasvasco',
    'espaa', 'badajoz', 'gipuzkoa', 'valencia', 'galicia'
]
for keyword in spain_state_repl:
    users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains(keyword)), 'location_country'] = 'spain'
# portugal
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('lisboa')), 'location_country'] = 'portugal'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('coimbra')), 'location_country'] = 'portugal'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('porto')), 'location_country'] = 'portugal'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('estremadura')), 'location_country'] = 'portugal'
# netherlands
netherlands_state_repl = [
    'noordholland', 'utrecht', 'zuidholland', 'overijssel', 'friesland', 'northholland', 'schleswigholstein',
    'zh', 'twente', 
]
for keyword in netherlands_state_repl:
    users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains(keyword)), 'location_country'] = 'netherlands'
# belgium
belgium_state_repl = [
    'vlaamsbrabant', 'liege'
]
for keyword in belgium_state_repl:
    users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains(keyword)), 'location_country'] = 'belgium'
# new zealand
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('auckland')), 'location_country'] = 'newzealand'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('northisland')), 'location_country'] = 'newzealand'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('waikato')), 'location_country'] = 'newzealand'
# italy
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('italia')), 'location_country'] = 'italy'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('toscana')), 'location_country'] = 'italy'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('piemonte')), 'location_country'] = 'italy'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('lombardia')), 'location_country'] = 'italy'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('gorizia')), 'location_country'] = 'italy'
users.loc[(users['location_country'] == 'null') & (users['location_state'] == 're'), 'location_country'] = 'italy'
# greece
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('townofbali')), 'location_country'] = 'greece'
# nigeria
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('imostate')), 'location_country'] = 'nigeria'
# southafrica
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('westerncape')), 'location_country'] = 'southafrica'
# romania
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('ilfov')), 'location_country'] = 'romania'
# malaysia
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('penang')), 'location_country'] = 'malaysia'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('negerisembilan')), 'location_country'] = 'malaysia'
# indonesia
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('jakarta')), 'location_country'] = 'indonesia'
# philippines
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('laguna')), 'location_country'] = 'philippines'
# singapore
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('singapore')), 'location_country'] = 'singapore'
# pakistan
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('punjab')), 'location_country'] = 'pakistan'
# denmark
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('jutland')), 'location_country'] = 'denmark'
# france
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('lorraine')), 'location_country'] = 'france'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('hautegaronne')), 'location_country'] = 'france'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('heraut')), 'location_country'] = 'france'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('rhnealpes')), 'location_country'] = 'france'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('iledefrance')), 'location_country'] = 'france'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('paca')), 'location_country'] = 'france'
# uruguay
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('montevideo')), 'location_country'] = 'uruguay'
# argentina
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('buenosaires')), 'location_country'] = 'argentina'
# peru
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('southamerica')), 'location_country'] = 'peru'
# chile
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('santiago')), 'location_country'] = 'chile'
# japan
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('iwakuni')), 'location_country'] = 'japan'
users.loc[(users['location_country'] == 'null') & (users['location_state'].str.contains('tokyo')), 'location_country'] = 'japan'



'''
location_city
'''
# usa
usa_city_repl = [
    'losang', 'seattle', 'sanf', 'sand', 'newyork', 'newark', 'newbedford', 'portland', 'cincinnati',
    'houston', 'albuquerque', 'chicago', 'austin', 'beaverton', 'raleigh', 'richmond', 'fairbanks',
    'minneapolis', 'stlouis', 'tucson', 'oakland', 'boston', 'kansascity', 'denver', 'springfield',
    'topeka', 'dallas', 'asheville', 'buffalo', 'fremont', 'stpaul', 'elcajon', 'miami', 'marysville',
    'baltimore', 'charleston', 'santamonica', 'knoxville', 'rochester', 'orlando', 'coloradosprings',
    'arlington', 'pensacola', 'sanjose', 'cedarrapids', 'olympia', 'lasvegas', 'mercerisland',
    'encinitas', 'omaha', 'lawrence', 'sacramento', 'norfolk', 'kirkwood', 'tallahassee', 'lexington',
    'kalamazoo', 'orleans', 'desmoines', 'aurora', 'annarbor', 'newbern', 'somerville', 'lakeland',
    'hartford', 'tigard', 'phoenix', 'irvine', 'sanantonio', 'mesa', 'brooklyn', 'philadelphia',
    'lacey', 'greenbay', 'pittsburg', 'wichita', 'elizabeth', 'murrieta', 'batonrouge', 'yuma',
    'baycity', 'lynchburg', 'santabarbara', 'statenisland', 'saintpaul', 'lakewood', 'fallschurch',
    'northhaven', 'frederick', 'milwaukie', 'cary', 'stcharles', 'lewiston', 'virginiabeach',
    'longbranch', 'indianapolis', 'portales', 'fountainvalley', 'sebastopol', 'washington', 'louisville',
    'millersburg'
]
for keyword in usa_city_repl:
    users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains(keyword)), 'location_country'] = 'usa'
# canada
canada_city_repl = [
    'calgary', 'vancouver', 'toronto', 'ottawa', 'fredericton', 'victoria', 'hamilton', 'montreal',
    'kelowna', 'winnipeg', 'saskatoon', 'halifax', 'edmonton', 'kitchener', 'regina', 'lethbridge',

]
for keyword in canada_city_repl:
    users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains(keyword)), 'location_country'] = 'canada'
# italy
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('milano')), 'location_country'] = 'italy'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('roma')), 'location_country'] = 'italy'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('rome')), 'location_country'] = 'italy'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('genova')), 'location_country'] = 'italy'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('torino')), 'location_country'] = 'italy'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('perugia')), 'location_country'] = 'italy'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('salerno')), 'location_country'] = 'italy'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('firenze')), 'location_country'] = 'italy'
# united kingdom
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('london')), 'location_country'] = 'united kingdom'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('manchester')), 'location_country'] = 'united kingdom'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('cambridge')), 'location_country'] = 'united kingdom'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('york')), 'location_country'] = 'united kingdom'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('birmingham')), 'location_country'] = 'united kingdom'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('edinburgh')), 'location_country'] = 'united kingdom'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('newcastle')), 'location_country'] = 'united kingdom'
# germany
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('hamburg')), 'location_country'] = 'germany'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('berlin')), 'location_country'] = 'germany'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('augsburg')), 'location_country'] = 'germany'
# france
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('paris')), 'location_country'] = 'france'
# spain
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('barcelona')), 'location_country'] = 'spain'
# finland
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('helsinki')), 'location_country'] = 'finland'
# australia
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('melbourne')), 'location_country'] = 'australia'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('sidney')), 'location_country'] = 'australia'
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('canberra')), 'location_country'] = 'australia'
# singapore
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('singapore')), 'location_country'] = 'singapore'

'''
모든 null 값을 다 볼 순 없으므로,
user_id 별 데이터 많은 순서대로 null 값 처리
'''
# usa
usa_uids = [
    83671, 179718, 187065, 104278, 146230, 93565, 67663, 84795, 175100,
    273190, 51350, 19493, 226745, 57620, 125031, 113663, 178201, 91631,
    83443, 239535, 135228, 23680, 259264, 209229, 929, 168036, 50129,
    129368, 136465, 8937, 84523, 241749, 48743, 132188, 270897, 171045,
    44842, 115473, 1131, 91017, 68768, 167587, 135411, 30889, 221557,
    39195, 154346, 273110, 29497, 223816, 38718, 175529, 186238, 239449,
    141543, 77676, 258277, 240113, 172486, 34988, 112818, 129474, 46295,
    142041, 268035, 176102, 126985, 93386, 114601, 30650, 24105, 170850,
    28372, 207651, 122802, 129389, 266764, 269140, 50504, 52993, 170208,
    162264, 45641, 226556, 241214

]
for uid in usa_uids:
    users.loc[users['user_id'] == uid, 'location_country'] = 'usa'    
# uk
users.loc[users['user_id'] == 178522, 'location_country'] = 'united kingdom'
users.loc[users['user_id'] == 5476, 'location_country'] = 'united kingdom'
users.loc[users['user_id'] == 237064, 'location_country'] = 'united kingdom'
users.loc[users['user_id'] == 241537, 'location_country'] = 'united kingdom'
# ireland
users.loc[users['user_id'] == 26432, 'location_country'] = 'ireland'
# canada
canada_uids = [44089, 79188, 176100, 34087, 172962, 103160, 206693]
for uid in canada_uids:
    users.loc[users['user_id'] == uid, 'location_country'] = 'canada'
# france
users.loc[users['user_id'] == 179641, 'location_country'] = 'france'
# germany
users.loc[users['user_id'] == 276538, 'location_country'] = 'germany'
users.loc[users['user_id'] == 102169, 'location_country'] = 'germany'
# austria
users.loc[users['user_id'] == 3923, 'location_country'] = 'austria'
users.loc[users['user_id'] == 14393, 'location_country'] = 'austria'
# portugal
users.loc[users['user_id'] == 164581, 'location_country'] = 'portugal'
# australia
users.loc[users['user_id'] == 11399, 'location_country'] = 'australia'
# malaysia
users.loc[users['user_id'] == 30445, 'location_country'] = 'malaysia'
users.loc[users['user_id'] == 28543, 'location_country'] = 'malaysia'
# philippines
users.loc[users['user_id'] == 131023, 'location_country'] = 'philippines'
#########################

In [13]:
users = users.drop(['location_city', 'location_state'], axis=1)

In [16]:
users.loc[users['location_country'] == 'null', 'location_country'] = 'empty'

In [ ]:
# user_id 별 책 권 수 count 열 추가
count = train.groupby("user_id").size()
dfcount = pd.DataFrame(count, columns=["count"])

users = users.merge(dfcount, on='user_id', how='left')
users = users.loc[users['count'].notna()]
users['count'] = users['count'].astype(int)

In [17]:
users.to_csv('../data/loc_users.csv', index=False)

In [22]:
books['language'].value_counts()

en       78823
de        1282
es        1017
fr         883
it         123
nl          67
pt          47
da          37
ca          23
ms          10
no           6
zh-CN        3
ja           3
gl           3
ru           3
la           3
el           1
th           1
ro           1
fa           1
eo           1
gd           1
ga           1
vi           1
zh-TW        1
ko           1
Name: language, dtype: int64

In [21]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149570 entries, 0 to 149569
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 149570 non-null  object 
 1   book_title           149570 non-null  object 
 2   book_author          149570 non-null  object 
 3   year_of_publication  149570 non-null  float64
 4   publisher            149570 non-null  object 
 5   img_url              149570 non-null  object 
 6   language             82343 non-null   object 
 7   category             80719 non-null   object 
 8   summary              82343 non-null   object 
 9   img_path             149570 non-null  object 
dtypes: float64(1), object(9)
memory usage: 11.4+ MB


In [20]:
books.head(1)

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg


In [17]:
pd.DataFrame(users.loc[users['age'].isna(), 'location_country'].value_counts())

,location_country
usa,20158
canada,3114
germany,1097
united kingdom,859
australia,469
...,...
trinidadandtobago,1
palau,1
guinea,1
jersey,1


In [18]:
users.loc[:, 'location_country'].value_counts()

usa               45128
canada             6446
germany            3574
united kingdom     3043
australia          1819
                  ...  
zambia                1
kosovo                1
tonga                 1
honduras              1
elsalvador            1
Name: location_country, Length: 142, dtype: int64

In [10]:
users.loc[(users['location_country'] == 'null') & (users['location_city'].str.contains('woodstock'))]#['location_state'].unique()

,user_id,age,location_city,location_state,location_country
4100,231251,NaN,woodstock,,null
7361,231252,NaN,woodstock,,null
51468,22510,NaN,woodstock,,null
52516,147646,23.0,woodstock,,null


In [104]:
#users.loc[users['location_country'].str.contains('phil')]

,user_id,age,location_city,location_state,location_country
919,36656,28.0,manila,na,philippines
1131,66428,16.0,dasmarias,cavite,philippines
1211,76301,19.0,pasigcity,,philippines
1280,86809,26.0,manila,metromanila,philippines
1291,88142,19.0,santiagocity,na,philippines
...,...,...,...,...,...
63539,169023,13.0,sanjuan,metromanila,philippines
64539,127510,NaN,quezoncity,metromanila,philippines
64648,211563,31.0,manila,na,philippines
65689,162937,19.0,valenciacity,,philippines


In [105]:
len(users['location_country'].unique())

142

In [17]:
train.head(1)

,user_id,isbn,rating
0,8,0002005018,4


In [23]:
train_df = train.merge(users, on='user_id', how='left').merge(books, on='isbn', how='left')
test_df = test.merge(users, on='user_id', how='left').merge(books, on='isbn', how='left')
train_df = train_df.drop(['location_city', 'location_state'], axis=1)
test_df = test_df.drop(['location_city', 'location_state'], axis=1)

In [24]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306795 entries, 0 to 306794
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              306795 non-null  int64  
 1   isbn                 306795 non-null  object 
 2   rating               306795 non-null  int64  
 3   age                  214133 non-null  float64
 4   location_country     306795 non-null  object 
 5   book_title           306795 non-null  object 
 6   book_author          306795 non-null  object 
 7   year_of_publication  306795 non-null  float64
 8   publisher            306795 non-null  object 
 9   img_url              306795 non-null  object 
 10  language             187711 non-null  object 
 11  category             185574 non-null  object 
 12  summary              187711 non-null  object 
 13  img_path             306795 non-null  object 
dtypes: float64(2), int64(2), object(10)
memory usage: 35.1+ MB


In [26]:
train_df['location_country'].value_counts()

usa               215445
canada             29129
united kingdom     12197
germany             9834
null                7031
                   ...  
zambia                 1
mozambique             1
niger                  1
algeria                1
paraguay               1
Name: location_country, Length: 138, dtype: int64

In [91]:
train_df.loc[train_df['location_country'] == 'null', 'user_id'].value_counts()[120:140]

/tmp/ipykernel_116744/870193709.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  train_df.loc[train_df['location_country'] == 'null', 'user_id'].value_counts()[120:140]


28543     6
206693    6
45641     6
226556    6
178458    6
241214    6
200009    6
233117    6
233928    6
265925    6
117390    6
11708     6
228913    6
129531    6
219729    6
26782     6
163194    5
105124    5
237797    5
38775     5
Name: user_id, dtype: int64

In [101]:
users.loc[users['user_id'] == 233117]

,user_id,age,location_city,location_state,location_country
4482,233117,NaN,didsbury,,null


In [100]:
train_df.loc[train_df['user_id'] == 200009]

,user_id,isbn,rating,age,location_city,location_state,location_country,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
63137,200009,0380701340,8,30.0,montevista,,null,Creed for the Third Millennium,Colleen McCullough,1986.0,Avon,http://images.amazon.com/images/P/0380701340.0...,en,['Fiction'],In a future America ravaged by wars and enduri...,images/0380701340.01.THUMBZZZ.jpg
82430,200009,0452281431,8,30.0,montevista,,null,Beauty's Punishment (Sleeping Beauty),A.N. Roquelaure,1984.0,Plume Books,http://images.amazon.com/images/P/0452281431.0...,NaN,NaN,NaN,images/0452281431.01.THUMBZZZ.jpg
96059,200009,034536676X,8,30.0,montevista,,null,The World According to Garp,John Irving,1994.0,Ballantine Books,http://images.amazon.com/images/P/034536676X.0...,en,['Fiction'],"T. S. Garp, a man with high ambitions for an a...",images/034536676X.01.THUMBZZZ.jpg
203370,200009,0061020613,9,30.0,montevista,,null,Witches Abroad,Terry Pratchett,2002.0,HarperTorch,http://images.amazon.com/images/P/0061020613.0...,en,['Fiction'],"But the road to Genua is bumpy, and along the ...",images/0061020613.01.THUMBZZZ.jpg
206957,200009,0061054909,2,30.0,montevista,,null,Love in Vein,Poppy Z. Brite,1995.0,Eos,http://images.amazon.com/images/P/0061054909.0...,en,['Fiction'],A sexy new repackaging of the extremely popula...,images/0061054909.01.THUMBZZZ.jpg
251291,200009,0688172849,9,30.0,montevista,,null,Test Pattern,Marjorie Klein,2000.0,William Morrow & Company,http://images.amazon.com/images/P/0688172849.0...,NaN,NaN,NaN,images/0688172849.01.THUMBZZZ.jpg


In [427]:
train_df.loc[train_df['location_country'] == 'france', 'language'].value_counts()

fr    849
en    498
de      6
es      6
nl      3
no      1
da      1
Name: language, dtype: int64

In [47]:
train_df[train_df['location_country'] == 'canada']['location_state']#.value_counts().head(20)

Series([], Name: location_state, dtype: object)

In [19]:
train_df['location_state'].value_counts().head(20)

 california          28888
 ontario             12979
 texas               12253
 new york            10557
 georgia             10537
 n/a                  9050
 florida              8868
 illinois             8393
 washington           8077
 pennsylvania         8032
 england              7370
 missouri             7326
                      6919
 ohio                 6761
 virginia             6716
 oregon               6207
 new jersey           5969
 michigan             5764
 british columbia     5360
 north carolina       5100
Name: location_state, dtype: int64

In [ ]:
train_df['']

In [27]:
books

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,['1940-1949'],"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,['Medical'],"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,['Fiction'],A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,['History'],"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg
...,...,...,...,...,...,...,...,...,...,...
149565,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987.0,Pocket Books,http://images.amazon.com/images/P/067161746X.0...,en,['Humor'],A tongue-in-cheek survival guide for single pe...,images/067161746X.01.THUMBZZZ.jpg
149566,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001.0,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,en,['Nature'],A daring twist on the travel-adventure genre t...,images/0767907566.01.THUMBZZZ.jpg
149567,0884159221,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985.0,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,NaN,NaN,NaN,images/0884159221.01.THUMBZZZ.jpg
149568,0912333022,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997.0,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,en,['Fiction'],These hilarious stories by the creator of publ...,images/0912333022.01.THUMBZZZ.jpg


In [ ]:
fig, ax = plt.subplots(3,5,figsize=(20,8))
for country, ax_ in zip(f['location_country'], ax.flatten()):
    users[(users['location_country']==country)]['age'].value_counts().sort_index().plot(ax=ax_, title=country)
plt.xlim(0,100)
plt.tight_layout()
plt.show()